In [39]:
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,Dense,Dropout,Conv1D,GlobalMaxPool1D,concatenate

In [40]:
#데이터 읽어오기
train_file = "./chatbot_data.csv"
data=pd.read_csv(train_file,delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()
#단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences=tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

MAX_SEQ_LEN =15  #단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences,maxlen=MAX_SEQ_LEN,padding='post')

#학습용, 검증용, 테스트용 데이터셋 생성
#학습셋:검증셋:테스트셋=7:2:1
ds=tf.data.Dataset.from_tensor_slices((padded_seqs,labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) *0.7)
val_size=int(len(padded_seqs)*0.2)
test_size=int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size +val_size).take(test_size).batch(20)

#하이퍼 파라미터 설정 
dropout_prob = 0.5
EMB_SIZE= 128
EPOCH= 5
VOCAB_SIZE = len(word_index) +1  #전체 단어 수

In [41]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128, 
    kernel_size=3, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128, 
    kernel_size=4, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128, 
    kernel_size=5, 
    padding='valid', 
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

c:\Users\KOREAVC\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [42]:
# 3,4,5 -gram 이후 합치기
concat = concatenate([pool1,pool2,pool3])

hidden =Dense(128, activation = tf.nn.relu)(concat)
dropout_hidden =Dropout(rate=dropout_prob)(hidden)
logits= Dense(3,name='logits')(dropout_hidden)
predictions = Dense(3,activation = tf.nn.softmax)(logits)

#모델 생성
model = Model(input_layer, outputs = predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#모델 학습
model.fit(train_ds, validation_data = val_ds, epochs=EPOCH,verbose=1)

#모델 평가(테스트 데이터셋 이용)
loss, accuracy= model.evaluate(test_ds,verbose=1)
print('Accuracy:%f' %(accuracy*100))
print('loss: %f'  %(loss))

#모델 저장
model.save('cnn_model.h5')

Epoch 1/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.4989 - loss: 0.9855 - val_accuracy: 0.6815 - val_loss: 0.6786
Epoch 2/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7516 - loss: 0.6216 - val_accuracy: 0.9133 - val_loss: 0.2797
Epoch 3/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8853 - loss: 0.3254 - val_accuracy: 0.9505 - val_loss: 0.1664
Epoch 4/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9400 - loss: 0.1983 - val_accuracy: 0.9695 - val_loss: 0.1048
Epoch 5/5
414/414 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9580 - loss: 0.1516 - val_accuracy: 0.9822 - val_loss: 0.0584
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9795 - loss: 0.0629


Accuracy:98.138750
loss: 0.070022


___________________________________________________________________________________________
#파일을 읽어와 CNN모델에 필요한 Q(질문)과 label(감정) 데이터를 features와 label 리스트에 저장한다

In [13]:
train_file ="./chatbot_data.csv"
data=pd.read_csv(train_file,delimiter=',')
features=data['Q'].tolist()
labels = data['label'].tolist()

In [20]:
#단어 인덱스 시퀀스 text_to_word_sequence() 함수 
corpus =[preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences= tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
#시퀀스 벡터 크기
MAX_SEQ_LEN = 15 
#패딩처리
padded_seqs= preprocessing.sequence.pad_sequences(sequences,maxlen=MAX_SEQ_LEN,padding='post')


In [22]:
#랜덤섞기
ds=tf.data.Dataset.from_tensor_slices((padded_seqs,labels))
ds = ds.shuffle(len(features))


In [23]:
#학습용,검증용,테스트용 7:2:1로 나눠 데이터셋 객체를 분리
train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds=ds.take(train_size).batch(20)
val_ds=ds.skip(train_size).take(val_size).batch(20)
test_ds=ds.skip(train_size + val_size).take(test_size).batch(2)

In [25]:
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length = MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate = dropout_prob)(embedding_layer)

______________________________________________________________________

In [26]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [27]:
#데이터 읽어보기
train_file="./chatbot_data.csv"
data=pd.read_csv(train_file,delimiter=',')
features= data['Q'].tolist()
labels = data['label'].tolist()

In [33]:
#단어 인덱스 시퀀스 벡터
corpus=[preprocessing.text.text_to_word_sequence(text) for text in features ]
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

MAX_SEQ_LEN =15 
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [34]:
#테스트용 데이터셋 생성
ds= tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds= ds.shuffle(len(features))
test_ds =ds.take(2000).batch(20) 

In [36]:
#감정 분류 CNN불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds,verbose =2)

TypeError: Error when deserializing class 'Dense' using config={'name': 'dense_3', 'trainable': True, 'dtype': 'float32', 'units': 3, 'activation': {'module': 'builtins', 'class_name': 'function', 'config': 'softmax_v2', 'registered_name': 'function'}, 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Could not interpret activation function identifier: {'module': 'builtins', 'class_name': 'function', 'config': 'softmax_v2', 'registered_name': 'function'}

In [43]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 데이터 읽어오기
train_file = "./chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

TypeError: Error when deserializing class 'Dense' using config={'name': 'dense_5', 'trainable': True, 'dtype': 'float32', 'units': 3, 'activation': {'module': 'builtins', 'class_name': 'function', 'config': 'softmax_v2', 'registered_name': 'function'}, 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Could not interpret activation function identifier: {'module': 'builtins', 'class_name': 'function', 'config': 'softmax_v2', 'registered_name': 'function'}

In [51]:
# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[1021])
print("단어 인덱스 시퀀스 : ", padded_seqs[1021])
print("문장 분류(정답) : ", labels[1021])


단어 시퀀스 :  ['놀러가고', '싶은데', '같이', '갈', '친구가', '없어']
단어 인덱스 시퀀스 :  [803 102  72 504  76  20   0   0   0   0   0   0   0   0   0]
문장 분류(정답) :  0


In [52]:
# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [1021]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
감정 예측 점수 :  [[9.8911196e-01 1.0075894e-02 8.1217493e-04]]
감정 예측 클래스 :  [0]
